# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
#pip install cartopy geoviews pyproj
import hvplot.pandas
import pandas as pd
import requests
 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,corner brook,48.9667,-57.9484,-1.02,92,96,1.25,CA,2025-03-23
1,1,nurota,40.5614,65.6886,13.04,88,80,3.76,UZ,2025-03-23
2,2,udachny,66.4167,112.4000,-20.38,99,99,2.95,RU,2025-03-23
3,3,adamstown,-25.0660,-130.1015,23.86,80,100,10.35,PN,2025-03-23
4,4,piti village,13.4626,144.6933,28.19,68,20,7.20,GU,2025-03-23


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,piti village,13.4626,144.6933,28.19,68,20,7.20,GU,2025-03-23
5,5,utrik,11.2278,169.8474,26.95,69,99,7.47,MH,2025-03-23
6,6,bardai,11.6500,7.2333,23.31,10,61,4.00,NG,2025-03-23
9,9,hobyo,5.3505,48.5268,28.30,78,19,5.74,SO,2025-03-23
10,10,sao felix do xingu,-6.6447,-51.9950,22.88,98,93,0.60,BR,2025-03-23
...,...,...,...,...,...,...,...,...,...,...
555,555,khash,28.2211,61.2158,18.73,34,74,2.42,IR,2025-03-23
557,557,isiro,2.7739,27.6160,18.70,100,100,1.04,CD,2025-03-23
559,559,pangoa,-12.1167,-73.0000,23.06,98,100,0.47,PE,2025-03-23
560,560,maragogi,-9.0122,-35.2225,26.44,73,10,3.67,BR,2025-03-23


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

/var/folders/p7/44v4jhr91plb0p3405m1crv80000gn/T/ipykernel_21098/3885843272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
4,piti village,GU,13.4626,144.6933,68,
5,utrik,MH,11.2278,169.8474,69,
6,bardai,NG,11.6500,7.2333,10,
9,hobyo,SO,5.3505,48.5268,78,
10,sao felix do xingu,BR,-6.6447,-51.9950,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
limit = 20 
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
piti village - nearest hotel: The Cliff Hotel
utrik - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
hobyo - nearest hotel: Iftin Hotel Hobyo
sao felix do xingu - nearest hotel: Hotel Terraço
bozoum - nearest hotel: No hotel found
suicheng - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
lompoc - nearest hotel: Red Roof Inn Lompoc
pisco - nearest hotel: La Portada
wajir - nearest hotel: Kilimanjaro
patos - nearest hotel: Hotel Zurique
huangpi - nearest hotel: 武汉黄陂华美达广场酒店
west island - nearest hotel: Cocos Village Bungalows
nagqu - nearest hotel: Naqu Hotel
gedo - nearest hotel: ጨሊያ ሆቴል
montijo - nearest hotel: Havanesa
kavaratti - nearest hotel: No hotel found
kamthi - nearest hotel: Shanti Nivas
ulimang - nearest hotel: M&A Riverside Bungalows
praia da vitoria - nearest hotel: Salles
misratah - nearest hotel: الفندق السياحى
sarakhs - nearest hotel: هتل ابریشم
huejutla de reyes - nearest hotel: Hotel Oviedo
sougueur - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
4,piti village,GU,13.4626,144.6933,68,The Cliff Hotel
5,utrik,MH,11.2278,169.8474,69,No hotel found
6,bardai,NG,11.6500,7.2333,10,No hotel found
9,hobyo,SO,5.3505,48.5268,78,Iftin Hotel Hobyo
10,sao felix do xingu,BR,-6.6447,-51.9950,98,Hotel Terraço
...,...,...,...,...,...,...
555,khash,IR,28.2211,61.2158,34,No hotel found
557,isiro,CD,2.7739,27.6160,100,Hôtel Magbetu
559,pangoa,PE,-12.1167,-73.0000,98,No hotel found
560,maragogi,BR,-9.0122,-35.2225,73,Pousada Mariluz


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)